In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 656 ms (started: 2021-08-26 13:44:52 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #4  Aug 26, 2021'

#git push origin master
git push

In [2]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[1] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 3.92 s (started: 2021-08-26 13:45:22 +08:00)


# keras预处理

Keras 预处理层 API 允许开发人员构建 Keras 原生输入处理管道。 这些输入处理管道可以在非 Keras 工作流中用作独立的预处理代码，直接与 Keras 模型结合，并作为 Keras SavedModel 的一部分导出。

使用 Keras 预处理层，您可以构建和导出真正端到端的模型：接受原始图像或原始结构化数据作为输入的模型； 自行处理特征归一化或特征值索引的模型。

# 可用的预处理

## 文本预处理
* `tf.keras.layers.experimental.preprocessingTextVectorization`：将原始字符串转换为可由嵌入层或密集层读取的编码表示。  

## 数值特征预处理
* `tf.keras.layers.experimental.preprocessing.Normalization`：执行输入特征的特征归一化。
* `tf.keras.layers.experimental.preprocessing.Discretization`：将连续的数值特征转化为整数分类特征。

## 分类特征预处理
* `tf.keras.layers.experimental.preprocessing.CategoryEncoding`：将整数分类特征转换为单热、多热或计数密集表示。
* `tf.keras.layers.experimental.preprocessing.Hashing`：执行分类特征散列，也称为“散列技巧”。
* `tf.keras.layers.experimental.preprocessing.StringLookup`：将字符串分类值转换为可由嵌入层或密集层读取的编码表示。
* `tf.keras.layers.experimental.preprocessing.IntegerLookup`：将整数分类值转换为可由嵌入层或密集层读取的编码表示。

## 图像预处理
这些层用于标准化图像模型的输入。
* tf.keras.layers.experimental.preprocessing.Resizing：将一批图像调整为目标尺寸。
* tf.keras.layers.experimental.preprocessing.Rescaling：重新缩放和偏移一批图像的值（例如，从 [0, 255] 范围内的输入到 [0, 1] 范围内的输入。
* tf.keras.layers.experimental.preprocessing.CenterCrop：返回一批图像的中心裁剪。

## 图像数据增强
这些层将随机增强变换应用于一批图像。 他们只在训练期间活跃。
* tf.keras.layers.experimental.preprocessing.RandomCrop
* tf.keras.layers.experimental.preprocessing.RandomFlip
* tf.keras.layers.experimental.preprocessing.RandomTranslation
* tf.keras.layers.experimental.preprocessing.RandomRotation
* tf.keras.layers.experimental.preprocessing.RandomZoom
* tf.keras.layers.experimental.preprocessing.RandomHeight
* tf.keras.layers.experimental.preprocessing.RandomWidth
* tf.keras.layers.experimental.preprocessing.RandomContrast

# `adapt()`方法

一些预处理层有一个内部状态，可以根据训练数据的样本进行计算。 有状态预处理层的列表是：
* `TextVectorization`：保存字符串标记和整数索引之间的映射
* `StringLookup` 和 `IntegerLookup`：保存输入值和整数索引之间的映射。
* 归一化：保存特征的均值和标准差。
* 离散化：保存有关值桶边界的信息。

至关重要的是，这些层是不可训练的。 他们的状态在训练期间没有设置； 它必须在训练之前设置，或者通过从预先计算的常量初始化它们，或者通过“适应”它们在数据上。

您可以通过 `adapt()` 方法将预处理层暴露给训练数据来设置它的状态：

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

data = np.array([[0.1, 0.2, 0.3], [0.8, 0.9, 1.0], [1.5, 1.6, 1.7],])
layer = layers.experimental.preprocessing.Normalization()
layer.adapt(data)
normalized_data = layer(data)

print("Features mean: %.2f" % (normalized_data.numpy().mean()))
print("Features std: %.2f" % (normalized_data.numpy().std()))

Features mean: -0.00
Features std: 1.00
time: 3.08 s (started: 2021-08-26 14:07:38 +08:00)


计算最后一个轴上每个索引的均值和方差。

In [30]:
adapt_data = np.array([[0., 7., 4.],
                       [2., 9., 6.],
                       [0., 7., 4.],
                       [2., 9., 6.]], dtype='float32')
input_data = np.array([[0., 7., 4.]], dtype='float32')

layer = layers.experimental.preprocessing.Normalization(axis=-1)
layer.adapt(adapt_data)
layer(input_data)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-1., -1., -1.]], dtype=float32)>

time: 217 ms (started: 2021-08-26 14:11:04 +08:00)


直接传递均值和方差。

In [32]:
input_data = np.array([[1.], [2.], [3.]], dtype='float32')
layer = layers.experimental.preprocessing.Normalization(mean=3., variance=2.)
layer(input_data)

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[-1.4142135 ],
       [-0.70710677],
       [ 0.        ]], dtype=float32)>

time: 39.7 ms (started: 2021-08-26 14:11:44 +08:00)


`adapt()` 方法采用 Numpy 数组或 `tf.data.Dataset` 对象。 在 `StringLookup` 和 `TextVectorization` 的情况下，您还可以传递字符串列表：

In [34]:
data = [
    "ξεῖν᾽, ἦ τοι μὲν ὄνειροι ἀμήχανοι ἀκριτόμυθοι",
    "γίγνοντ᾽, οὐδέ τι πάντα τελείεται ἀνθρώποισι.",
    "δοιαὶ γάρ τε πύλαι ἀμενηνῶν εἰσὶν ὀνείρων:",
    "αἱ μὲν γὰρ κεράεσσι τετεύχαται, αἱ δ᾽ ἐλέφαντι:",
    "τῶν οἳ μέν κ᾽ ἔλθωσι διὰ πριστοῦ ἐλέφαντος,",
    "οἵ ῥ᾽ ἐλεφαίρονται, ἔπε᾽ ἀκράαντα φέροντες:",
    "οἱ δὲ διὰ ξεστῶν κεράων ἔλθωσι θύραζε,",
    "οἵ ῥ᾽ ἔτυμα κραίνουσι, βροτῶν ὅτε κέν τις ἴδηται.",
]
layer = layers.experimental.preprocessing.TextVectorization()
layer.adapt(data)
vectorized_text = layer(data)
print(vectorized_text)

tf.Tensor(
[[37 12 25  5  9 20 21  0  0]
 [51 34 27 33 29 18  0  0  0]
 [49 52 30 31 19 46 10  0  0]
 [ 7  5 50 43 28  7 47 17  0]
 [24 35 39 40  3  6 32 16  0]
 [ 4  2 15 14 22 23  0  0  0]
 [36 48  6 38 42  3 45  0  0]
 [ 4  2 13 41 53  8 44 26 11]], shape=(8, 9), dtype=int64)
time: 91.3 ms (started: 2021-08-26 14:13:33 +08:00)


此外，自适应层总是公开一个选项，通过构造函数参数或权重分配直接设置状态。 如果预期状态值在层构建时已知，或者是在 `adapt()` 调用之外计算的，则可以在不依赖层的内部计算的情况下设置它们。 例如，如果 `TextVectorization`、`StringLookup` 或 `IntegerLookup` 层的外部词汇文件已经存在，则可以通过在层的构造函数参数中传递词汇文件的路径，将这些文件直接加载到查找表中。

这是我们使用预先计算的词汇表实例化 `StringLookup` 层的示例：

In [36]:
vocab = ["a", "b", "c", "d"]
data = tf.constant([["a", "c", "d"], ["d", "z", "b"]])
layer = layers.experimental.preprocessing.StringLookup(vocabulary=vocab)
vectorized_data = layer(data)
print(vectorized_data)

tf.Tensor(
[[2 4 5]
 [5 1 3]], shape=(2, 3), dtype=int64)
time: 7.73 ms (started: 2021-08-26 14:14:56 +08:00)


# 在模型之前或模型内部预处理数据

您可以通过两种方式使用预处理层：

**选项 1：** 使它们成为模型的一部分，如下所示：

In [ ]:
inputs = keras.Input(shape=input_shape)
x = preprocessing_layer(inputs)
outputs = rest_of_the_model(x)
model = keras.Model(inputs, outputs)

使用此选项，预处理将在设备上进行，与模型执行的其余部分同步，这意味着它将受益于 GPU 加速。 如果您在 GPU 上训练，这是标准化层以及所有图像预处理和数据增强层的最佳选择。

**选项 2**：将其应用到您的 `tf.data.Dataset`，从而获得生成批量预处理数据的数据集，如下所示：

In [ ]:
dataset = dataset.map(lambda x, y: (preprocessing_layer(x), y))

使用此选项，您的预处理将在 CPU 上异步进行，并在进入模型之前进行缓冲。 此外，如果您在数据集上调用 `dataset.prefetch(tf.data.AUTOTUNE)`，预处理将与训练并行有效地进行：

In [ ]:
dataset = dataset.map(lambda x, y: (preprocessing_layer(x), y))
dataset = dataset.prefetch(tf.data.AUTOTUNE)
model.fit(dataset, ...)

这是 `TextVectorization` 和所有结构化数据预处理层的最佳选择。 如果您在 CPU 上训练并且使用图像预处理层，它也是一个不错的选择。

在 TPU 上运行时，您应该始终在 `tf.data` 管道中放置预处理层（`Normalization` 和 `Rescaling` 除外，它们在 TPU 上运行良好，并且通常用作第一层是图像模型）。

# 在推理时在模型内部进行预处理的好处

即使您选择选项 2，您以后也可能希望导出一个仅包含预处理层的推理端到端模型。这样做的主要好处是它使您的模型具有可移植性，并且有助于减少训练/服务偏差。

当所有数据预处理都是模型的一部分时，其他人可以加载和使用您的模型，而无需知道每个特征是如何被编码和规范化的。您的推理模型将能够处理原始图像或原始结构化数据，并且不需要模型的用户了解例如用于文本的标记化方案、用于分类特征的索引方案、图像像素值是否标准化为 [-1, +1] 或 [0, 1] 等。如果您要导出模型，这将特别强大到另一个运行时，例如 TensorFlow.js：您不必在 JavaScript 中重新实现预处理管道。

如果您最初将预处理层放在 `tf.data` 管道中，则可以导出打包预处理的推理模型。只需实例化一个链接预处理层和训练模型的新模型：

In [ ]:
inputs = keras.Input(shape=input_shape)
x = preprocessing_layer(inputs)
outputs = training_model(x)
inference_model = keras.Model(inputs, outputs)

# 快速参考
## 图像数据增强
请注意，图像数据增强层仅在训练期间处于活动状态（类似于 Dropout 层）。

In [37]:
from tensorflow import keras
from tensorflow.keras import layers

# 创建具有水平翻转、旋转、缩放功能的数据增强阶段
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1),
    ]
)

# 加载一些数据
(x_train, y_train), _ = keras.datasets.cifar10.load_data()
input_shape = x_train.shape[1:]
classes = 10

# 创建增强图像（及其标签）的 tf.data 管道
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.batch(16).map(lambda x, y: (data_augmentation(x), y))


# 创建模型并在增强图像数据上对其进行训练
inputs = keras.Input(shape=input_shape)
x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)  # Rescale inputs
outputs = keras.applications.ResNet50(  # Add the rest of the model
    weights=None, input_shape=input_shape, classes=classes
)(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")
model.fit(train_dataset, steps_per_epoch=5)

5/5 [==============================] - 33s 97ms/step - loss: 8.4553


time: 37.7 s (started: 2021-08-26 14:22:28 +08:00)


您可以从头开始在示例图像分类中看到类似的设置。

## 归一化数值特征

In [38]:
# Load some data
(x_train, y_train), _ = keras.datasets.cifar10.load_data()
x_train = x_train.reshape((len(x_train), -1))
input_shape = x_train.shape[1:]
classes = 10

# 创建一个归一化层并使用训练数据设置其内部状态
normalizer = layers.experimental.preprocessing.Normalization()
normalizer.adapt(x_train)

# 创建一个包含规范化层的模型
inputs = keras.Input(shape=input_shape)
x = normalizer(inputs)
outputs = layers.Dense(classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

# 训练模型
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
model.fit(x_train, y_train)

1563/1563 [==============================] - 14s 9ms/step - loss: 2.1306


time: 33.8 s (started: 2021-08-26 14:27:06 +08:00)


## 通过 one-hot 编码对字符串分类特征进行编码

In [41]:
# Define some toy data
data = tf.constant([["a"], ["b"], ["c"], ["b"], ["c"], ["a"]])

# 使用 StringLookup 构建特征值的索引并对输出进行编码。
lookup = layers.experimental.preprocessing.StringLookup(output_mode="binary")
lookup.adapt(data)

# 转换新的测试数据（包括未知特征值）
test_data = tf.constant([["a"], ["b"], ["c"], ["d"], ["e"], [""]])
encoded_data = lookup(test_data)
print(encoded_data)

tf.Tensor(
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]], shape=(6, 4), dtype=float32)
time: 117 ms (started: 2021-08-26 15:21:02 +08:00)


请注意，这里，索引 0 保留用于词汇外值（在adapter() 期间未看到的值）。

您可以在从头开始的结构化数据分类示例中看到 StringLookup 的作用。

## 通过 one-hot 编码对整数分类特征进行编码

In [42]:
# Define some toy data
data = tf.constant([[10], [20], [20], [10], [30], [0]])

# Use IntegerLookup to build an index of the feature values and encode output.
lookup = layers.experimental.preprocessing.IntegerLookup(output_mode="binary")
lookup.adapt(data)

# Convert new test data (which includes unknown feature values)
test_data = tf.constant([[10], [10], [20], [50], [60], [0]])
encoded_data = lookup(test_data)
print(encoded_data)

tf.Tensor(
[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]], shape=(6, 4), dtype=float32)
time: 102 ms (started: 2021-08-26 15:23:44 +08:00)


请注意，索引 0 保留用于缺失值（您应该将其指定为值 0），而索引 1 保留用于词汇表外值（在adapter() 期间未看到的值）。 您可以使用 IntegerLookup 的 mask_token 和 oov_token 构造函数参数来配置它。

您可以从头开始在示例结构化数据分类中看到 IntegerLookup 的运行情况。

## 将散列技巧应用于整数分类特征

如果您的分类特征可以采用许多不同的值(大约为 $10^3$ 或更高)，其中每个值仅在数据中出现几次，那么对特征值进行索引和单热编码变得不切实际且无效。 相反，应用“散列技巧”可能是一个好主意：将值散列到固定大小的向量。 这使特征空间的大小保持可控，并消除了对显式索引的需要。

In [44]:
# 样本数据：10,000 个随机整数，值在 0 到 100,000 之间
data = np.random.randint(0, 100000, size=(10000, 1))

# 使用 Hashing 层将值散列到范围 [0, 64]
hasher = layers.experimental.preprocessing.Hashing(num_bins=64, salt=1337)

# 使用 CategoryEncoding 层对散列值进行多热编码
encoder = layers.experimental.preprocessing.CategoryEncoding(num_tokens=64, output_mode="binary")
encoded_data = encoder(hasher(data))
print(encoded_data.shape)

(10000, 64)
time: 47.9 ms (started: 2021-08-26 15:27:27 +08:00)


In [50]:
encoded_data[:2]

<tf.Tensor: shape=(2, 64), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)>

time: 2.86 ms (started: 2021-08-26 15:28:16 +08:00)


## 将文本编码为标记索引序列

这是您应该如何预处理要传递到嵌入层的文本。

In [52]:
# 定义一些文本数据来适应图层
adapt_data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)

# 创建一个 TextVectorization 层
text_vectorizer = layers.experimental.preprocessing.TextVectorization(output_mode="int")
# 通过`adapt()`索引词汇表
text_vectorizer.adapt(adapt_data)

# 试试图层
print(
    "Encoded text:\n", text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
)

Encoded text:
 [[ 2 19 14  1  9  2  1]]
time: 91.5 ms (started: 2021-08-26 15:31:16 +08:00)


In [54]:
text_vectorizer.vocabulary_size()

22

time: 1.33 ms (started: 2021-08-26 15:32:14 +08:00)


In [55]:
# 创建一个简单的模型
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(input_dim=text_vectorizer.vocabulary_size(), output_dim=16)(inputs)
x = layers.GRU(8)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

# 创建一个带标签的数据集（包括未知标记）
train_dataset = tf.data.Dataset.from_tensor_slices(
    (["The Brain is deeper than the sea", "for if they are held Blue to Blue"], [1, 0])
)

#预处理字符串输入，将它们转换为 int 序列
train_dataset = train_dataset.batch(2).map(lambda x, y: (text_vectorizer(x), y))
# 在 int 序列上训练模型
print("\nTraining model...")
model.compile(optimizer="rmsprop", loss="mse")
model.fit(train_dataset)


Training model...
1/1 [==============================] - 3s 3s/step - loss: 0.5171


time: 2.9 s (started: 2021-08-26 15:34:02 +08:00)


In [56]:
# 对于推理，您可以导出接受字符串作为输入的模型
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
outputs = model(x)
end_to_end_model = keras.Model(inputs, outputs)

# Call the end-to-end model on test data (which includes unknown tokens)
print("\nCalling end-to-end model on test string...")
test_data = tf.constant(["The one the other will absorb"])
test_output = end_to_end_model(test_data)
print("Model output:", test_output)


Calling end-to-end model on test string...
Model output: tf.Tensor([[0.02545869]], shape=(1, 1), dtype=float32)
time: 330 ms (started: 2021-08-26 15:35:13 +08:00)


您可以在示例文本分类中从头开始看到 `TextVectorization` 层与嵌入模式相结合的效果。

请注意，在训练此类模型时，为了获得最佳性能，您应始终使用 `TextVectorization` 层作为输入管道的一部分。

## 使用多热编码将文本编码为 ngram 的密集矩阵
这就是您应该如何预处理要传递到 Dense 层的文本。

In [57]:
# 定义一些文本数据来适应图层
adapt_data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# 使用“binary” output_mode 和 ngrams=2 实例化 TextVectorization（索引所有双字组）
text_vectorizer = layers.experimental.preprocessing.TextVectorization(output_mode="binary", ngrams=2)
# 通过`adapt()`索引二元组
text_vectorizer.adapt(adapt_data)

# 试试图层
print(
    "Encoded text:\n", text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
)

Encoded text:
 [[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]]
time: 102 ms (started: 2021-08-26 15:44:03 +08:00)


In [58]:
# 创建一个简单的模型
inputs = keras.Input(shape=(text_vectorizer.vocabulary_size(),))
outputs = layers.Dense(1)(inputs)
model = keras.Model(inputs, outputs)

# 创建一个带标签的数据集（包括未知标记）
train_dataset = tf.data.Dataset.from_tensor_slices(
    (["The Brain is deeper than the sea", "for if they are held Blue to Blue"], [1, 0])
)

# 预处理字符串输入，将它们转换为 int 序列
train_dataset = train_dataset.batch(2).map(lambda x, y: (text_vectorizer(x), y))
# 在 int 序列上训练模型
print("\nTraining model...")
model.compile(optimizer="rmsprop", loss="mse")
model.fit(train_dataset)


Training model...
1/1 [==============================] - 0s 379ms/step - loss: 1.6035


time: 619 ms (started: 2021-08-26 15:44:43 +08:00)


In [59]:
# 对于推理，您可以导出接受字符串作为输入的模型
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
outputs = model(x)
end_to_end_model = keras.Model(inputs, outputs)

# 在测试数据（包括未知令牌）上调用端到端模型
print("\nCalling end-to-end model on test string...")
test_data = tf.constant(["The one the other will absorb"])
test_output = end_to_end_model(test_data)
print("Model output:", test_output)


Calling end-to-end model on test string...
Model output: tf.Tensor([[-0.1585224]], shape=(1, 1), dtype=float32)
time: 90.1 ms (started: 2021-08-26 15:44:52 +08:00)


## 使用 TF-IDF 加权将文本编码为 ngram 的密集矩阵
这是在将文本传递到 Dense 层之前预处理文本的另一种方法。

In [61]:
#Define some text data to adapt the layer
adapt_data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with "tf-idf" output_mode
# (binary with TF-IDF weighting) and ngrams=2 (index all bigrams)
text_vectorizer = layers.experimental.preprocessing.TextVectorization(output_mode="tf-idf", ngrams=2)
# Index the bigrams and learn the TF-IDF weights via `adapt()`
text_vectorizer.adapt(adapt_data)

# Try out the layer
print(
    "Encoded text:\n", text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
)

Encoded text:
 [[5.461647  1.6945957 0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        1.0986123 1.0986123 1.0986123 0.        0.
  0.        0.        0.        0.        0.        0.        0.
  1.0986123 0.        0.        0.        0.        0.        0.
  0.        1.0986123 1.0986123 0.        0.        0.       ]]
time: 118 ms (started: 2021-08-26 15:46:44 +08:00)


In [62]:
# Create a simple model
inputs = keras.Input(shape=(text_vectorizer.vocabulary_size(),))
outputs = layers.Dense(1)(inputs)
model = keras.Model(inputs, outputs)

# Create a labeled dataset (which includes unknown tokens)
train_dataset = tf.data.Dataset.from_tensor_slices(
    (["The Brain is deeper than the sea", "for if they are held Blue to Blue"], [1, 0])
)

# Preprocess the string inputs, turning them into int sequences
train_dataset = train_dataset.batch(2).map(lambda x, y: (text_vectorizer(x), y))
# Train the model on the int sequences
print("\nTraining model...")
model.compile(optimizer="rmsprop", loss="mse")
model.fit(train_dataset)


Training model...
1/1 [==============================] - 0s 380ms/step - loss: 4.6367


time: 619 ms (started: 2021-08-26 15:46:59 +08:00)


In [63]:
# For inference, you can export a model that accepts strings as input
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
outputs = model(x)
end_to_end_model = keras.Model(inputs, outputs)

# Call the end-to-end model on test data (which includes unknown tokens)
print("\nCalling end-to-end model on test string...")
test_data = tf.constant(["The one the other will absorb"])
test_output = end_to_end_model(test_data)
print("Model output:", test_output)


Calling end-to-end model on test string...
Model output: tf.Tensor([[-0.02850113]], shape=(1, 1), dtype=float32)
time: 110 ms (started: 2021-08-26 15:47:01 +08:00)


# 重要问题

## 使用具有非常大词汇量的查找层
您可能会发现自己在 `TextVectorization`、`StringLookup` 层或 `IntegerLookup` 层中使用非常大的词汇表。 通常，大于 500MB 的词汇表将被视为“非常大”。

在这种情况下，为了获得最佳性能，您应该避免使用 `adapt()`。 相反，预先计算您的词汇量（您可以为此使用 Apache Beam 或 TF Transform）并将其存储在文件中。 然后在构建时通过传递文件路径作为词汇表参数将词汇表加载到层中。

## 在 TPU pod 上或通过 ParameterServerStrategy 使用查找层。
在 TPU pod 上或通过 ParameterServerStrategy 在多台机器上训练时，使用 `TextVectorization`、`StringLookup` 或 `IntegerLookup` 层时，有一个突出的问题会导致性能下降。 这将在 TensorFlow 2.7 中修复。